In [ ]:
%matplotlib inline

import matplotlib
from pydub import AudioSegment
from scipy import signal,stats
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt
import numpy as np
from pydub import playback
from pydub import generators
import array
import time
import musdb
from torch.utils import data
import math
import Net



device = torch.device("cuda:0")
mus = musdb.DB(root_dir="/mnt/scratch/lts2/laly/musdb")



######################################################################################################################  
    

def DANN_train(track):

    estimates = {}

    net = torch.load('DANN.pt')
    
    net.to(device)

    fs = 44100/5
    
    track_i = np.array(track.audio)
    track_v = np.array(track.targets['vocals'].audio)
    track_d = np.array(track.targets['drums'].audio)
    track_b = np.array(track.targets['bass'].audio)
    track_o = np.array(track.targets['other'].audio)
    
    track_i = (track_i[:,0]+track_i[:,1])/2
    track_v = (track_v[:,0]+track_v[:,1])/2
    track_d = (track_d[:,0]+track_d[:,1])/2
    track_b = (track_b[:,0]+track_b[:,1])/2
    track_o = (track_o[:,0]+track_o[:,1])/2
    
    track_i = signal.resample(track_i,len(track_i)//5,window = 'hamming')
    track_v = signal.resample(track_v,len(track_v)//5,window = 'hamming')
    track_d = signal.resample(track_d,len(track_d)//5,window = 'hamming')
    track_b = signal.resample(track_b,len(track_b)//5,window = 'hamming')
    track_o = signal.resample(track_o,len(track_o)//5,window = 'hamming') 
    
    f, t, Sxx_noised = signal.stft(track_i, fs, 'hamming',2048,1024,2048)
    
    Sxx_real_noised = np.real(Sxx_noised)*100000
    Sxx_imag_noised = np.imag(Sxx_noised)*100000
    
    z_output_vocal = [[0 for l in range(len(Sxx_real_noised[0]))] for k in range(1025)]
    z_o = torch.Tensor(z_output_vocal)
    z_output_vocal = z_o.detach().numpy()
    z_output_drums = [[0 for l in range(len(Sxx_real_noised[0]))] for k in range(1025)]
    z_o = torch.Tensor(z_output_drums)
    z_output_drums = z_o.detach().numpy()
    z_output_bass = [[0 for l in range(len(Sxx_real_noised[0]))] for k in range(1025)]
    z_o = torch.Tensor(z_output_bass)
    z_output_bass = z_o.detach().numpy()
    
    criterion = nn.L1Loss()
    criterion_GPU = criterion.to(device)

    for i in range(0,len(Sxx_real_noised[0]),88):
        
        Sxx_real_noised_i = Sxx_real_noised[:,i:i+88]
        Sxx_imag_noised_i = Sxx_imag_noised[:,i:i+88]

        Sxx_torch_noised = (torch.Tensor([[Sxx_real_noised_i,Sxx_imag_noised_i]]))

        if Sxx_torch_noised.size()[3] == 88:
           
            Sxx_torch_GPU_noised = Sxx_torch_noised.to(device)
            
            outputs_GPU = net(Sxx_torch_GPU_noised)
            
            outputs = outputs_GPU.cpu()
            outputs = outputs.detach().numpy()
            Sxx_torch_noised = Sxx_torch_noised.detach().numpy()
            outputs = [[Sxx_torch_noised[0][0]*outputs[0][0],Sxx_torch_noised[0][1]*outputs[0][1],Sxx_torch_noised[0][0]*outputs[0][2],Sxx_torch_noised[0][1]*outputs[0][3],Sxx_torch_noised[0][0]*outputs[0][4],Sxx_torch_noised[0][1]*outputs[0][5]]]  
            
            z_output_vocal = z_output_vocal.astype(complex)
            z_output_vocal[:,i:i+88] = (outputs[0][0] + 1j*outputs[0][1])/50000
            z_output_drums = z_output_drums.astype(complex)
            z_output_drums[:,i:i+88] = (outputs[0][2] + 1j*outputs[0][3])/20000
            z_output_bass = z_output_bass.astype(complex)
            z_output_bass[:,i:i+88] = (outputs[0][4] + 1j*outputs[0][5])/20000
    
    t, track_output_vocal = signal.istft(z_output_vocal,fs,'hamming',2048,1024,2048)
    t, track_output_drums = signal.istft(z_output_drums,fs,'hamming',2048,1024,2048)
    t, track_output_bass = signal.istft(z_output_bass,fs,'hamming',2048,1024,2048)
    
    track_output_vocal = signal.resample(track_output_vocal, len(track_output_vocal)*5,window = 'hamming')
    track_output_drums = signal.resample(track_output_drums, len(track_output_drums)*5,window = 'hamming')
    track_output_bass = signal.resample(track_output_bass, len(track_output_bass)*5,window = 'hamming')
    track_v = signal.resample(track_v,len(track_v)*5,window = 'hamming')
    track_d = signal.resample(track_d,len(track_d)*5,window = 'hamming')
    track_b = signal.resample(track_b,len(track_b)*5,window = 'hamming')
    track_o = signal.resample(track_o,len(track_o)*5,window = 'hamming')
    
    track_output_vocal = track_output_vocal[0:len(track_v)]
    track_d = track_d[0:len(track_output_vocal)]
    track_b = track_b[0:len(track_output_vocal)]
    track_o = track_o[0:len(track_output_vocal)]
    
    estimates = {
        track.name + '_vocals' : track_output_vocal,
        track.name + '_drums' : track_output_drums, 
        track.name + '_bass' : track_output_bass, 
    }
    
    return estimates


mus.run(CNN_train, subsets="test", estimates_dir=None) #estimates_dir="laly@lts2gdk0:/mnt/scratch/lts2/laly_test" if you want to listen to the output songs

  0%|          | 0/50 [00:00<?, ?it/s]

James Elder & Mark M Thompson - The English Actor
Load OK
np.array OK
stereoToMono OK
Avant STFT
Après STFT


/home/laly/alligator/local/lib/python3.5/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
